In [247]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [248]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY")
LANGSMITH_PROJECT=os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_ENDPOINT"]="https://api.smith.langchain.com"

In [249]:
LANGSMITH_PROJECT

'Langchain-agent'

In [250]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [251]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")

In [252]:
llm.invoke("Hi,how are you?")

AIMessage(content="As an AI, I don't have feelings, but I'm here and ready to assist you! How can I help you today? 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 15, 'total_tokens': 48, 'completion_time': 0.06, 'prompt_time': 0.001915076, 'queue_time': 0.235110162, 'total_time': 0.061915076}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b9710c2d-41a7-4bb9-a58e-a84785255446-0', usage_metadata={'input_tokens': 15, 'output_tokens': 33, 'total_tokens': 48})

Predefine tools

In [253]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [254]:
api_wrapper=WikipediaAPIWrapper()

In [255]:
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [256]:
tool.name

'wikipedia'

In [257]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [258]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [259]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Retrieval-augmented generation
Summary: Retrieval-augmented generation (RAG) is a technique that enables generative artificial intelligence (Gen AI) models to retrieve and incorporate new information. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to supplement information from its pre-existing training data. This allows LLMs to use domain-specific and/or updated information.  Use cases include providing chatbot access to internal company data or generating responses based on authoritative sources.
RAG improves lar

In [260]:
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults()
tool.invoke({"query": "who is Rohit Sharma?"})

[{'url': 'https://www.britannica.com/biography/Rohit-Sharma',
  'content': 'Rohit Sharma (born April 30, 1987, Bansod, India) is an Indian international cricketer and the current captain of the Indian cricket team in the Test and One-Day International (ODI) formats. Sharma was also the captain of India’s Twenty20 International team until he announced his retirement on June 29, 2024, after leading the Indian side to its T20I World Cup victory. He captained the Mumbai Indians, a franchise team in the Indian Premier League (IPL), for 10 years and led them to five titles. [...] Sharma was named the ICC Men’s ODI Cricketer of the Year in 2019 and was included in the ICC Men’s ODI Team of the Year in 2014 and in each year from 2016 to 2019. He was also included in the ICC Men’s Test Team of the Year in 2021, mainly for his achievements on India’s Test tour of England that year. For the same achievements, the Wisden Cricketer’s Almanack named him one of the five Wisden Cricketers of the Year 

Create a Custom tool

In [261]:
from langchain.agents import tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [262]:
get_word_length.invoke("Cricket")

7

In [264]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [266]:
@tool
def summisation(a: int, b: int) -> int:
    """Adding two numbers."""
    return a + b

In [267]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [268]:

print(summisation.name)
print(summisation.description)
print(summisation.args)

summisation
Adding two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [269]:
multiply.invoke({"a":10,"b":20})

200

In [270]:
summisation.invoke({"a":10,"b":20})

30

Concept of Agents

In [271]:
from langchain.agents import AgentType
from langchain.agents import load_tools 
from langchain.agents import initialize_agent

In [272]:
tool=load_tools(["wikipedia"],llm=llm)

In [273]:
agent=initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [274]:
agent.run("what is llama and who create this llm model?")



> Entering new AgentExecutor chain...
Thought: I need to search Wikipedia for "LLaMA" to find information about it and its creators. 
Action: wikipedia
Action Input: LLaMA (large language model)
Observation: Page: Llama (language model)
Summary: Llama (Large Language Model Meta AI, formerly stylized as LLaMA) is a family of large language models (LLMs) released by Meta AI starting in February 2023. The latest version is Llama 3.3, released in December 2024.
Llama models are trained at different parameter sizes, ranging between 1B and 405B. Initially  only a foundation model, starting with Llama 2, Meta AI released instruction fine-tuned versions alongside foundation models.
Model weights for the first version of Llama were only available to researchers on a case-by-case basis, under a non-commercial license. Unauthorized copies of the first model were shared via BitTorrent. Subsequent versions of Llama were made accessible outside academia and released under licenses that permitted s

'LLaMA is a family of large language models created by Meta AI.'

A Tool Calling Agent refers to an AI system or framework that dynamically invokes external tools or APIs to accomplish tasks as part of its decision-making process. Instead of relying solely on internal reasoning or knowledge, the agent calls specific tools to perform actions like calculations, searches, or other operations. This type of agent is often used in real-world applications where tasks require interaction with external systems.

In [275]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [276]:
search = TavilySearchResults()

In [277]:
search.invoke("what is the weather in ohio")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'Ohio', 'region': 'Illinois', 'country': 'USA United States of America', 'lat': 41.6026, 'lon': -89.4525, 'tz_id': 'America/Chicago', 'localtime_epoch': 1743440830, 'localtime': '2025-03-31 12:07'}, 'current': {'last_updated_epoch': 1743440400, 'last_updated': '2025-03-31 12:00', 'temp_c': 6.2, 'temp_f': 43.2, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.6, 'wind_kph': 18.7, 'wind_degree': 344, 'wind_dir': 'NNW', 'pressure_mb': 1017.0, 'pressure_in': 30.02, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 61, 'cloud': 25, 'feelslike_c': 2.7, 'feelslike_f': 36.9, 'windchill_c': 5.7, 'windchill_f': 42.3, 'heatindex_c': 8.8, 'heatindex_f': 47.8, 'dewpoint_c': 0.4, 'dewpoint_f': 32.7, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 4.6, 'gust_mph': 13.4, 'gust_kph': 21.5}}"},
 {'url': 'https://ocj.com/2025/03/ohio-ag-weather-and-f

In [278]:
tools = [search]

In [279]:
from langchain import hub

In [280]:
prompt=hub.pull("hwchase17/openai-functions-agent")

In [281]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [282]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

**agent_scratchpad:** The agent does not call a tool just once to get the desired answer, but has a structure that calls tools repeatedly until the desired answer is obtained. Each time you call a tool, what the previous call was like, information about the previous call, and the result are stored in this field.

In [283]:
from langchain.agents import create_tool_calling_agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [284]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [285]:
agent_executor.invoke({"input": "hello how are you?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'hello how are you'}`


[{'url': 'https://en.wikipedia.org/wiki/Hello,_How_Are_You', 'content': '"The Music Goes \'Round My Head"(1967) | "Hello, How Are You"(1968) | "Good Times"(1968)\n"The Music Goes \'Round My Head"(1967) | "Hello, How Are You"(1968) | "Land of Make Believe"(1968)\n"Falling Off the Edge of the World"(1967) | "Hello, How Are You"(1968) | "Gonna Have a Good Time (Good Times)"(1968)\n"Hello, How Are You" is a 1968 song and single by Australian  rock group the Easybeats, which was written by band members George Young and Harry Vanda.\n\nBackground [...] "Hello, How Are You"\nCover of the 1968 Netherlands single\nSinglebyThe Easybeats\nfrom the albumVigil\nB-side | "Falling Off the Edge of the World" [UK & Australia]"Come In You\'ll Get Pneumonia" [US]\nReleased | March 1968\nRecorded | Pye Studios,London, 1968\nGenre | Pop\nLength | 3:17\nLabel | Parlophone(Australia),United

{'input': 'hello how are you?', 'output': ''}

In [286]:
agent_executor.invoke({"input": "whats the weather in new york?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in new york'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.7142, 'lon': -74.0064, 'tz_id': 'America/New_York', 'localtime_epoch': 1743440893, 'localtime': '2025-03-31 13:08'}, 'current': {'last_updated_epoch': 1743440400, 'last_updated': '2025-03-31 13:00', 'temp_c': 17.2, 'temp_f': 63.0, 'is_day': 1, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/day/122.png', 'code': 1009}, 'wind_mph': 13.0, 'wind_kph': 20.9, 'wind_degree': 217, 'wind_dir': 'SW', 'pressure_mb': 1007.0, 'pressure_in': 29.73, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 75, 'cloud': 100, 'feelslike_c': 17.2, 'feelslike_f': 63.0, 'windchill_c': 18.7, 'windchill_f': 65.7, 'heatindex_c': 20.3, 'heatindex_f': 68.6, 'dewpoint_c': 14.8, 'dewpoint_f': 58.6, 'vis_km': 14.0, 'vi

{'input': 'whats the weather in new york?',
 'output': '```json\n{"tool_call_id": "call_4344", "type": "function", "function": {"name": "tavily_search_results_json", "parameters": {"query": "weather in new york city"}\n```\n\n\n\n'}

1. RAG tool
2. ReAct
3. custom tool with ReAct agent
4. agent code from latest versions(v0.2 and v0.3)

In [287]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [288]:
loader = WebBaseLoader("https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/")
docs = loader.load()

In [289]:
docs

[Document(metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content="\n\n\nLangGraph Studio: The first agent IDE\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph Studio: The first agent IDE\nLangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.\n\n4 min read\nAug 1, 2024\n\n\n\n\n\nLLMs have paved the way for the dev

In [290]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

In [291]:
vector = FAISS.from_documents(documents,embeddings)
retriever = vector.as_retriever()

In [292]:
retriever.invoke("what is langgraph studio?")[0]

Document(id='7e6a436c-6499-45f2-b797-c2e1344019d3', metadata={'source': 'https://blog.langchain.dev/langgraph-studio-the-first-agent-ide/', 'title': 'LangGraph Studio: The first agent IDE', 'description': 'LangGraph Studio provides a specialized agent IDE for visualizing, interacting with, and debugging complex agentic applications. See how to use it on your desktop today.', 'language': 'en'}, page_content="0:00\n\n                            /0:20\n\n\n1×\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nConclusionBuilding agentic applications differs from traditional software development. While code editors remain important, new IDEs designed for agents are also needed. LangGraph Studio is a step in this direction, and we're excited to see how it enhances your workflow.For more on LangGraph Studio, check out our documentation. You can also watch a video walkthrough on YouTube if that's more your style. You can\xa0sign up for LangSmith\xa0today to try out LangGraph Studio for free.We'd also love your feed

In [293]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langgraph_studio",
    "Search for information about LangGraph. For any questions about LangGraph, you must use this tool!",
)

In [294]:
tools = [search, retriever_tool]

In [295]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [296]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [297]:
agent_executor.invoke({"input": "hi! what is a langgraph studio and how to use it?"})



> Entering new AgentExecutor chain...

Invoking: `langgraph_studio` with `{'query': 'what is langgraph studio and how to use it'}`


0:00

                            /0:20


1×














ConclusionBuilding agentic applications differs from traditional software development. While code editors remain important, new IDEs designed for agents are also needed. LangGraph Studio is a step in this direction, and we're excited to see how it enhances your workflow.For more on LangGraph Studio, check out our documentation. You can also watch a video walkthrough on YouTube if that's more your style. You can sign up for LangSmith today to try out LangGraph Studio for free.We'd also love your feedback - drop us a line at hello@langchain.dev or on Twitter to share your thoughts.


Join our newsletter
Updates from the LangChain team and community


Enter your email

Subscribe

Processing your application...
Success! Please check your inbox and click the link to confirm your subscription.
Sorry, 

{'input': 'hi! what is a langgraph studio and how to use it?',
 'output': "LangGraph Studio is a new type of IDE designed specifically for building agentic applications.  It's currently a desktop app for Apple Silicon, but support for other platforms is coming soon. You can download and try it for free with a LangSmith account. \n\n\n\nLangGraph Studio helps you visualize, interact with, and debug complex agent graphs, making it easier to understand and work with them. \n"}

In [299]:
agent_executor.invoke({"input": "whats the weather in ohio?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in ohio'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Ohio', 'region': 'Illinois', 'country': 'USA United States of America', 'lat': 41.6026, 'lon': -89.4525, 'tz_id': 'America/Chicago', 'localtime_epoch': 1743441019, 'localtime': '2025-03-31 12:10'}, 'current': {'last_updated_epoch': 1743440400, 'last_updated': '2025-03-31 12:00', 'temp_c': 6.2, 'temp_f': 43.2, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 11.6, 'wind_kph': 18.7, 'wind_degree': 344, 'wind_dir': 'NNW', 'pressure_mb': 1017.0, 'pressure_in': 30.02, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 61, 'cloud': 25, 'feelslike_c': 2.7, 'feelslike_f': 36.9, 'windchill_c': 5.7, 'windchill_f': 42.3, 'heatindex_c': 8.8, 'heatindex_f': 47.8, 'dewpoint_c': 0.4, 'dewpoint_f': 32.7, 'vis_km': 16.0, 'vis_mil

{'input': 'whats the weather in ohio?',
 'output': 'The current temperature in Ohio is 6.2 degrees Celsius or 43.2 degrees Fahrenheit. The weather is partly cloudy. \n'}

*ReAct Agent (Reasoning + Acting):*
Definition:

* The ReAct framework combines reasoning and acting in a single loop to handle tasks. The agent uses natural language reasoning (thinking through steps) and task actions (performing tasks like calculations or data retrieval).

* The agent uses a combination of reasoning steps to guide actions in real-time, using feedback from those actions to further inform the next step in reasoning.

How it works:

* Step 1: The agent receives a question or task.
* Step 2: It reasons aloud (in natural language) about how to solve it.
* Step 3: Based on its reasoning, it takes actions (e.g., searching a database, calculating something).
* Step 4: The results of these actions are integrated into its reasoning and may trigger further actions.
* Step 5: It repeats the process until it arrives at a solution.
  

Key points:

* Combines thinking and doing (reasoning and actions).
* Performs iterative steps, updating its process based on action results.
* Typically handles complex decision-making scenarios.
* 
Example:

* Original task: “Calculate the total number of apples in a basket if there are 4 baskets and 7 apples in each.”
* Reasoning: "I need to multiply 4 by 7 to get the total number of apples."
* Action: Perform the multiplication.
* Result: "There are 28 apples."
* Reasoning: "I am done."

In [301]:
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

In [302]:
google_search = GoogleSerperAPIWrapper()

In [303]:
tools = [
    Tool(
        name="Web Answer",
        func=google_search.run,
        description="useful for when you need to ask with search",
        verbose=True
    )
]

In [305]:
template = '''Answer the following questions as best you can. You have access to the following tools:
{tools}
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!
Question: {input}
Thought:{agent_scratchpad}'''

In [306]:
prompt = PromptTemplate.from_template(template)

In [307]:
search_agent = create_react_agent(llm,tools,prompt)

In [308]:
agent_executor = AgentExecutor(
    agent=search_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
)

In [310]:
agent_executor.invoke({"input": "Where is the hometown of the 2007 T20 Cricket World cup caption winner and his score?"})



> Entering new AgentExecutor chain...
Thought: I need to find out who won the 2007 T20 Cricket World Cup. Then I need to find the hometown of that team's captain and their score in the final. 
Action: Web Answer
Action Input: Who won the 2007 T20 Cricket World Cup?IND vs PAK ; Series result, India won the 2007/08 ICC World Twenty20 ; Match number, T20I no. 46 ; Hours of play (local time), First Session 14:00-15:20, Interval ... India won the tournament, beating Pakistan in the final at Johannesburg. 2007 ICC World Twenty20. Tournament logo. Dates, 11 – ... Final, Johannesburg, September 24, 2007, ICC World Twenty20, IND Flag IND 157/5, PAK Flag PAK (19.3/20 ov, T:158) 152, India won by 5 runs. Three teams, West Indies (2012, 2016), England (2010, 2022) and India (2007, 2024), have won the competition twice each. Pakistan (2009), Sri Lanka (2014), and ... The Indian men's cricket team won the first T20 World Cup held in South Africa in 2007 while England are the inaugural World Twenty

{'input': 'Where is the hometown of the 2007 T20 Cricket World cup caption winner and his score?',
 'output': 'The hometown of the 2007 T20 Cricket World Cup winning captain MS Dhoni is Ranchi, Jharkhand.',
 'intermediate_steps': [(AgentAction(tool='Web Answer', tool_input='Who won the 2007 T20 Cricket World Cup?', log="Thought: I need to find out who won the 2007 T20 Cricket World Cup. Then I need to find the hometown of that team's captain and their score in the final. \nAction: Web Answer\nAction Input: Who won the 2007 T20 Cricket World Cup?"),
   "IND vs PAK ; Series result, India won the 2007/08 ICC World Twenty20 ; Match number, T20I no. 46 ; Hours of play (local time), First Session 14:00-15:20, Interval ... India won the tournament, beating Pakistan in the final at Johannesburg. 2007 ICC World Twenty20. Tournament logo. Dates, 11 – ... Final, Johannesburg, September 24, 2007, ICC World Twenty20, IND Flag IND 157/5, PAK Flag PAK (19.3/20 ov, T:158) 152, India won by 5 runs. Thr

In [311]:
agent_executor.invoke({"input": "who is a god of cricket what is highest score of the god of cricket?"})



> Entering new AgentExecutor chain...
Thought: I need to find out who is considered the "God of Cricket" and then find their highest score.  
Action: Web Answer
Action Input: Who is considered the God of Cricket?Fondly called the “God of Cricket” or “Master Blaster,” Sachin Tendulkar is considered one of the greatest batsmen of all time in the sport that is considered a national institution in India, cricket.Thought: I know Sachin Tendulkar is the God of Cricket, now I need to find his highest score. 
Action: Web Answer
Action Input: What is Sachin Tendulkar's highest score in cricket?Sachin Tendulkar: Cricketer and former Member of Rajya Sabha. Sachin Ramesh Tendulkar is an Indian former international cricketer who captained the Indian national team. Sachin Tendulkar Born: April 24, 1973 (age 51 years), Mumbai, India. Sachin Tendulkar Number of centuries: 100. Sachin Tendulkar Previous office: Member of Rajya Sabha (2012–2018). Sachin Tendulkar Dates joined: 2008 (Mumbai Indians), 1

{'input': 'who is a god of cricket what is highest score of the god of cricket?',
 'output': "Sachin Tendulkar's highest score in cricket is 248 not out.",
 'intermediate_steps': [(AgentAction(tool='Web Answer', tool_input='Who is considered the God of Cricket?', log='Thought: I need to find out who is considered the "God of Cricket" and then find their highest score.  \nAction: Web Answer\nAction Input: Who is considered the God of Cricket?'),
   'Fondly called the “God of Cricket” or “Master Blaster,” Sachin Tendulkar is considered one of the greatest batsmen of all time in the sport that is considered a national institution in India, cricket.'),
  (AgentAction(tool='Web Answer', tool_input="What is Sachin Tendulkar's highest score in cricket?", log="Thought: I know Sachin Tendulkar is the God of Cricket, now I need to find his highest score. \nAction: Web Answer\nAction Input: What is Sachin Tendulkar's highest score in cricket?"),
   "Sachin Tendulkar: Cricketer and former Member o

ReAct with Custom tool

In [317]:
# Custom tool for the Agent 
from langchain.agents import tool
@tool
def get_employee_id(name):
  """
  To get employee id, it takes employee name as arguments
  name(str): Name of the employee
  """
  fake_employees = {
    "Alice": "E001",
    "Bob": "E002",
    "Charlie": "E003",
    "Diana": "E004",
    "Evan": "E005",
    "Fiona": "E006",
    "George": "E007",
    "Hannah": "E008",
    "Ian": "E009",
    "Jasmine": "E010"}
  
  return fake_employees.get(name,"Employee not found")

In [318]:
# Custom tool for the Agent 
@tool
def get_employee_salary(employee_id):
  """
  To get the salary of an employee, it takes employee_id as input and return salary
  """
  employee_salaries = {
    "E001": 56000,
    "E002": 47000,
    "E003": 52000,
    "E004": 61000,
    "E005": 45000,
    "E006": 58000,
    "E007": 49000,
    "E008": 53000,
    "E009": 50000,
    "E010": 55000
    }
  return employee_salaries.get(employee_id,"Employee not found")

In [319]:
# Saved React Prompt in langchain hub, we could manually type the prompt as well.
prompt = hub.pull("hwchase17/react")

In [320]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [321]:
tools = [get_employee_salary, get_employee_id]

In [322]:
agent = create_react_agent(llm,tools,prompt)

In [323]:
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [324]:
agent_executor.invoke({"input":"What is the Salary of Evan?"})



> Entering new AgentExecutor chain...
Thought: I need to find Evan's employee ID first, then use that to get his salary.
Action: get_employee_id
Action Input: EvanE005Thought: I now have Evan's employee ID, I can use it to get his salary.
Action: get_employee_salary
Action Input: E00545000Thought: I now know the final answer
Final Answer: 45000 


> Finished chain.


{'input': 'What is the Salary of Evan?', 'output': '45000'}

In [325]:
agent_executor.invoke({"input":"What is the Salary of Divakar?"})



> Entering new AgentExecutor chain...
Thought: I need to find Divakar's employee ID first, then I can use that to get his salary.
Action: get_employee_id
Action Input: DivakarEmployee not foundQuestion: What is the Salary of Divakar?
Thought: I need to find Divakar's employee ID first, then I can use that to get his salary.
Action: get_employee_id
Action Input: DivakarEmployee not foundQuestion: What is the Salary of Divakar?
Thought: I need to find Divakar's employee ID first, then I can use that to get his salary.
Action: get_employee_id
Action Input: DivakarEmployee not foundThought: It seems Divakar is not an employee in the system. 
Final Answer: Employee not found 




> Finished chain.


{'input': 'What is the Salary of Divakar?', 'output': 'Employee not found'}

*Self-Ask with Search Agent:*

Definition:

This approach allows the AI to ask itself follow-up questions when it doesn't immediately know the answer. It performs a recursive questioning process to break down complex queries into smaller, more manageable ones.
Once the AI generates these follow-up questions, it performs an external search (e.g., through Google or an internal database) to gather the necessary information before formulating a response.
How it works:

* Step 1: Receive a complex question.
* Step 2: The agent identifies sub-questions or follow-up questions.
* Step 3: It performs a search or fetches answers to these questions from external resources (like a web search).
* Step 4: The answers are aggregated to provide a complete response.
  
Key points:

Emphasizes question decomposition.
Relies on external search for sub-questions.
Useful for answering open-ended or broad questions where the answer is not immediately available.

Example:

* Original question: “How many moons does Jupiter have?”
* Sub-question: “What is Jupiter?” (search)
* Sub-question: “What are moons?” (search)
* Finally, it retrieves the answer: "Jupiter has 79 moons."

In [326]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

In [327]:
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_community.tools.tavily_search import TavilyAnswer

In [328]:
tools = [
    Tool(
        name="Intermediate Answer",
        func=google_search.run,
        description="useful for when you need to ask with search",
        verbose=True
    )
]

In [329]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/self-ask-with-search")

In [332]:
print(prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [333]:
# Construct the Self Ask With Search Agent
agent = create_self_ask_with_search_agent(llm, tools, prompt)

In [334]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True)

In [335]:
agent_executor.invoke(
    {"input": "which asian country has largest population?"}
)



> Entering new AgentExecutor chain...
Yes.
Follow up: Which countries are located in Asia?There are 48 countries in Asia today, according to the United Nations. The full list is shown in the table below, with current population and subregion (based ... Asia is subdivided into 49 countries, five of them (Georgia, Azerbaijan, Russia, Kazakhstan and Turkey) are transcontinental countries lying partly in Europe. Asia: Being the biggest continent in the world, Asia includes 50 independent countries and occupies the eastern part of the single Eurasian landmass. Armenia · Bahrain · Brunei · Cambodia · China · Cyprus · Georgia · Indonesia · Iraq · Israel · Japan · Jordan · Kuwait · Kyrgyzstan · Laos · Lebanon · Malaysia ... How many countries are there in Asia? The area is shared by 50 countries. By far the largest country in terms of area is Russia, with more than 17 million ... Asia, the world's largest and most diverse continent. It occupies the eastern four-fifths of the giant Eurasian l

{'input': 'which asian country has largest population?',
 'output': 'India (or China, depending on the source and date).  The intermediate answers conflict a bit as to whether India or China is most populous, and the population numbers shift over time.'}

In [337]:
agent_executor.invoke(
    {"input": "who is current caption of indian cricket team?"}
)



> Entering new AgentExecutor chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 24
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 21
}
]